# Embedding 构建

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import math
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

In [2]:
random_torch_test = torch.rand(3, 2)
random_torch_test

tensor([[0.0203, 0.2529],
        [0.8478, 0.4440],
        [0.6016, 0.4999]])

In [3]:
from torch import Tensor

# 将输入的词汇表索引转换为指定维度的embedding形式
class TokenEmbedding(nn.Embedding):
    def __init__(self, vocab_size, embedding_dim):
        """
        初始化 TokenEmbedding 类的实例。

        参数:
        vocab_size (int): 词汇表的大小，即词汇表中唯一词元的数量。
        embedding_dim (int): 每个词元对应的嵌入向量的维度。
        """
        # 调用父类 nn.Embedding 的构造函数，初始化词嵌入层
        # vocab_size: 词汇表的大小，确定嵌入矩阵的行数
        # embedding_dim: 嵌入向量的维度，确定嵌入矩阵的列数
        # padding_idx=1: 指定填充词元的索引为 1，该索引对应的嵌入向量将被初始化为全零且不会被训练更新
        super(TokenEmbedding, self).__init__(vocab_size, embedding_dim, padding_idx=1)

In [30]:
# 构建位置编码embedding
class PositionEmbedding(nn.Module):
    """
    位置编码模块，用于为输入序列添加位置信息。
    在Transformer架构中，由于模型本身不具备捕捉序列顺序的能力，
    因此需要通过位置编码来引入序列中元素的位置信息。

    Attributes:
        device (torch.device): 计算设备，如 'cpu' 或 'cuda'。
        encoding (torch.Tensor): 位置编码矩阵，形状为 (max_len, embedding_dim)。
    """
    def __init__(self, embedding_dim, max_len, device):
        """
        初始化位置编码模块。

        Args:
            embedding_dim (int): 嵌入维度，即每个位置编码向量的长度。
            max_len (int): 最大序列长度，即位置编码矩阵的最大行数。
            device (torch.device): 计算设备，如 'cpu' 或 'cuda'。
        """
        super(PositionEmbedding, self).__init__()
        self.device = device
        # 初始化位置编码矩阵，形状为 (max_len, embedding_dim)，初始值全为0
        self.encoding = torch.zeros(max_len, embedding_dim, device = self.device)
        # 位置编码不需要计算梯度，因为它是固定的
        self.encoding.requires_grad = False
        # 生成位置索引，形状为 (max_len, 1), 转换为浮点型， 并扩展为 (max_len, embedding_dim)一个二维张量
        pos = torch.arange(0, max_len, device=device).float().unsqueeze(dim = 1)
        # 生成偶数索引，用于计算正弦和余弦值，形状为 (embedding_dim // 2,)
        _2i = torch.arange(0, embedding_dim, step=2, device=device).float()
        # 计算位置编码的偶数维度，使用正弦函数
        self.encoding[:, 0::2] = torch.sin(pos / (10000 ** (_2i / embedding_dim)))
        # 计算位置编码的奇数维度，使用余弦函数
        self.encoding[:, 1::2] = torch.cos(pos / (10000 ** (_2i / embedding_dim)))
        

    def forward(self, x):
        """
        前向传播方法，根据输入序列的长度截取相应的位置编码。

        Args:
            x (torch.Tensor): 输入序列，形状为 (batch_size, seq_len)。

        Returns:
            torch.Tensor: 截取后的位置编码，形状为 (seq_len, embedding_dim)。
        """
        batch_size, seq_len = x.size()
        # 根据输入序列的长度截取相应的位置编码
        return self.encoding[:seq_len, :]



In [31]:
class TransformerEmbedding(nn.Module):
    def __init__(self, vocab_size, embedding_dim, max_len, dropout_prob, device):
        super(TransformerEmbedding, self).__init__()    
        self.token_embedding = TokenEmbedding(vocab_size, embedding_dim)
        self.position_embedding = PositionEmbedding(embedding_dim, max_len, device)
        # 防止过拟合
        self.drop_out = nn.Dropout(p = dropout_prob) 

    def forward(self, x):
        token_embedding = self.token_embedding(x)
        position_embedding = self.position_embedding(x)
        return self.drop_out(token_embedding + position_embedding)


In [33]:
vocab_size = 1000
embedding_dim = 512
max_len = 128
dropout_prob = 0.1
device = 'cpu'
# device = 'cuda'

test_embedding = TransformerEmbedding(
    vocab_size = vocab_size, 
    embedding_dim = embedding_dim, 
    max_len = max_len, 
    dropout_prob = dropout_prob, 
    device = device
)

# 按照序列的规定长度拼接一个张量 进入 embedding模块进行embedding构建
test_input = torch.randint(0, 1000, (1, max_len))
print("测试输入：", test_input)
test_output = test_embedding.forward(test_input)
print("测试输出：", test_output)
print("测试输出维度：", test_output.shape)
# 预期输出形状应为 (1, 128, 512)
# 1: batch_size（批次大小）
# 128: sequence_length（序列长度）
# 512: embedding_dim（嵌入维度）


测试输入： tensor([[280, 699, 171,   2, 738, 544, 688, 318, 195, 761,  64, 166, 795, 645,
         330, 538, 627, 608, 932, 291, 980, 329, 385, 207, 562, 944, 576, 731,
         967, 812, 645, 878, 428,   5, 167, 296, 548, 618, 347, 783, 190, 849,
         102,   5, 328, 709, 947, 505, 419, 247, 154, 812, 673, 206, 572, 213,
         119, 744, 539, 567, 525, 144, 389, 417, 721, 209, 846, 932, 442, 637,
         999, 847,  25, 747, 515, 314, 532, 672, 484, 683, 943, 685, 228,  38,
         571,  98, 206, 138, 145, 808, 230, 320, 896, 748, 691, 316, 518, 726,
         632, 278, 339, 499, 783, 824, 920,   4, 107, 781, 336,   8, 999, 360,
         721, 870, 427, 334, 197, 133, 453, 139, 553, 214, 894, 886, 649, 123,
         740, 462]])
测试输出： tensor([[[-0.0000,  0.5848, -0.2572,  ...,  0.4867,  2.0487,  0.0000],
         [ 0.9114,  0.6461,  0.5768,  ...,  1.2462, -0.5499,  0.0000],
         [ 1.1638, -0.5677,  0.7471,  ...,  1.1358,  0.0000,  1.1008],
         ...,
         [-0.0000,  2.2345,  